In [1]:
from selenium import webdriver
from pyshadow.main import Shadow
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import re
import time
import string
#Inputting english 5 letter words
dictionary = "american-english.txt"
#Assiging letters the letters
letters = set(string.ascii_letters)
noAttempt = 6
#Setting max length of word to 5 letters
lenWord = 5

In [2]:
#creating an array with all the words
wordlist = [word.strip() for word in open(dictionary, "r").readlines()]
#new array with only 5 letter words
tryWords = {
    word.lower()
    for word in wordlist    
    if len(word) == lenWord and set(word) < letters
    }
tryWords

{'outgo',
 'twist',
 'pleat',
 'joist',
 'solve',
 'snack',
 'cabby',
 'inner',
 'dogma',
 'vague',
 'break',
 'shoal',
 'pooch',
 'apple',
 'motor',
 'sepia',
 'scold',
 'affix',
 'lathe',
 'media',
 'gonad',
 'untie',
 'trash',
 'grasp',
 'baton',
 'cabin',
 'utter',
 'junta',
 'rearm',
 'miner',
 'child',
 'after',
 'spoof',
 'close',
 'haute',
 'jaunt',
 'blood',
 'hover',
 'cling',
 'piney',
 'whisk',
 'cargo',
 'joust',
 'anode',
 'ratty',
 'swamp',
 'antic',
 'guest',
 'exist',
 'entry',
 'squad',
 'wrong',
 'heavy',
 'quite',
 'slush',
 'learn',
 'zebra',
 'jumpy',
 'flake',
 'karma',
 'crone',
 'heron',
 'pagan',
 'totem',
 'phone',
 'brood',
 'field',
 'chock',
 'timer',
 'azure',
 'miser',
 'genie',
 'power',
 'width',
 'grown',
 'crown',
 'rhino',
 'cobra',
 'awful',
 'flail',
 'visor',
 'steal',
 'fishy',
 'edict',
 'pithy',
 'wound',
 'beady',
 'aloud',
 'graph',
 'wight',
 'swear',
 'wreak',
 'trice',
 'befit',
 'thief',
 'crepe',
 'query',
 'thumb',
 'crest',
 'woozy',


In [3]:
from collections import Counter
from itertools import chain

#counts the number of times a letter appears in the whole array
iterLetter = Counter(chain.from_iterable(tryWords))

iterLetter


Counter({'o': 754,
         'u': 467,
         't': 729,
         'g': 311,
         'w': 195,
         'i': 671,
         's': 669,
         'p': 367,
         'l': 719,
         'e': 1233,
         'a': 979,
         'j': 27,
         'v': 153,
         'n': 575,
         'c': 477,
         'k': 210,
         'b': 281,
         'y': 425,
         'r': 899,
         'd': 393,
         'm': 316,
         'h': 389,
         'f': 230,
         'x': 37,
         'q': 29,
         'z': 40})

In [4]:
#total number of letters in the list
total = len(tryWords)*lenWord
#divides the number of times a letter appears by the total number of letters
freqLetter = {character: value / total 
                    for character, value in iterLetter.items()}

freqLetter

{'o': 0.0651403887688985,
 'u': 0.04034557235421166,
 't': 0.06298056155507559,
 'g': 0.026868250539956805,
 'w': 0.016846652267818573,
 'i': 0.057969762419006476,
 's': 0.057796976241900645,
 'p': 0.031706263498920084,
 'l': 0.06211663066954644,
 'e': 0.10652267818574514,
 'a': 0.08457883369330453,
 'j': 0.002332613390928726,
 'v': 0.013218142548596112,
 'n': 0.04967602591792657,
 'c': 0.04120950323974082,
 'k': 0.01814254859611231,
 'b': 0.02427645788336933,
 'y': 0.0367170626349892,
 'r': 0.07766738660907127,
 'd': 0.033952483801295896,
 'm': 0.027300215982721383,
 'h': 0.033606911447084234,
 'f': 0.019870410367170625,
 'x': 0.0031965442764578834,
 'q': 0.002505399568034557,
 'z': 0.0034557235421166306}

In [5]:
def calculate_word_commonality(word):
    #intializes score to be 0
    score = 0
    for char in word:
        #add the freuqency of each character in the word to the score
        score += freqLetter[char]
    #returns the total score divided by 1
    return score / (lenWord - len(set(word)) + 1)

In [6]:
import operator

def sort_by_word_commonality(words):
    #sorts the words into descending order with highest score first
    sort_by = operator.itemgetter(1)
    return sorted(
        [(word, calculate_word_commonality(word)) for word in words],
        key = sort_by,
        reverse=True,
    )

def display_word_table(word_commonalities):
    #prints the words score
    for (word,freq) in word_commonalities:
        print(f"{word:<10} | {freq:<5.2}")


In [7]:
#rudementary method when scraping wordle didnt work
def input_word():
    while True:
        word = input("Whad'ya put in> ")
        if len(word) == lenWord and word.lower() in tryWords:
            break
    return word.lower()

def input_response():
    print("Put the following based on the response from Wordle:")
    print(" G for Green")
    print(" Y for Yellow")
    print(" ? for Gray")
    while True:
        response = input("Response from Wordle> ")
        if len(response) == lenWord and set(response) <= {"G", "Y", "?"}:
            break
        else:
            print(f"Error - invalid answer {response}")
    return response

In [8]:
def match_word_vector(word, word_vector):
    #they same length
    assert len(word) == len(word_vector)
    for letter, v_letter in zip(word, word_vector):
        #removing words with grey letters
        if letter not in v_letter:
            return False
    return True

def match(word_vector, possible_words):
    return [word for word in possible_words if match_word_vector(word, word_vector)]
    #returns possible words

In [9]:
def enter_word(browser, word):
    shadow = Shadow(browser)
    wordleBot = browser.find_element_by_tag_name('html')
    #finds the answer box
    wordleBot.click()
    time.sleep(1)
    wordleBot.send_keys(word)
    wordleBot.send_keys(Keys.ENTER)
    #autotypes and clicks enter
    time.sleep(1)
    finalEvaluation = list(word)
    #inputs the chosen word
    gameRow = shadow.find_element("game-row[letters="+word+"]")
    for char in set(word):
        gameTile = shadow.find_elements(gameRow,"game-tile[letter=" + char +"]")
        index = -1
        for elem in gameTile:
            if elem.get_attribute("evaluation") == 'present':
                finalEvaluation[word.index(char, index + 1, len(word))] = 'Y' 
            if elem.get_attribute("evaluation") == 'correct':
                finalEvaluation[word.index(char, index + 1, len(word))] = 'G'
            if elem.get_attribute("evaluation") == 'absent':
                finalEvaluation[word.index(char, index + 1, len(word))] = '?'
            index = word.index(char, index + 1, len(word))
    return finalEvaluation

In [21]:
def solve():
    option = webdriver.ChromeOptions()
    option.binary_location = '/opt/brave.com/brave/brave'
    browser = webdriver.Chrome(executable_path= '/home/yui/Documents/GitHub/yuiWordle/chromedriver', options=option)
    shadow = Shadow(browser)
    browser.get('http://www.powerlanguage.co.uk/wordle/')
    time.sleep(1)
    possible_words = tryWords.copy()
    word_vector = [set(string.ascii_lowercase) for _ in range(lenWord)]
    yLetters = set()
    for attempt in range(1, noAttempt + 1):
        #limits it to 6 tries
        print(f"Attempt {attempt} with {len(possible_words)} possible words")
        sortedWords = sort_by_word_commonality(possible_words)
        display_word_table(sortedWords[:15])
        i = 0
        word = sortedWords[0][0]
        #chosen word is the first in the sorted list hence one with the highest value
        while yLetters.issubset(set(word)) == False:
            i = i + 1
            word = sortedWords[i][0]
            #choses next best word with no greys
        response = enter_word(browser,word)
        for idx, letter in enumerate(response):
            if letter == "G":
                word_vector[idx] = {word[idx]}
            elif letter == "Y":
                try:
                    word_vector[idx].remove(word[idx])
                    yLetters.add(word[idx])
                except KeyError:
                    pass
            elif letter == "?":
                for vector in word_vector:
                    try:
                        vector.remove(word[idx])
                    except KeyError:
                        pass
        possible_words = match(word_vector, possible_words)
    

In [17]:
def solve():
    option = webdriver.ChromeOptions()
    option.binary_location = '/opt/brave.com/brave/brave'
    browser = webdriver.Chrome(executable_path= '/home/yui/Documents/GitHub/yuiWordle/chromedriver', options=option)
    shadow = Shadow(browser)
    browser.get('https://wordle.berknation.com/')
    time.sleep(3)
    possible_words = tryWords.copy()
    word_vector = [set(string.ascii_lowercase) for _ in range(lenWord)]
    yLetters = set()
    for attempt in range(1, noAttempt + 1):
        print(f"Attempt {attempt} with {len(possible_words)} possible words")
        sortedWords = sort_by_word_commonality(possible_words)
        display_word_table(sortedWords[:15])
        i = 0
        word = sortedWords[0][0]
        while yLetters.issubset(set(word)) == False:
            i = i + 1
            word = sortedWords[i][0]
        response = enter_word(browser,word)
        for idx, letter in enumerate(response):
            if letter == "G":
                word_vector[idx] = {word[idx]}
            elif letter == "Y":
                try:
                    word_vector[idx].remove(word[idx])
                    yLetters.add(word[idx])
                except KeyError:
                    pass
            elif letter == "?":
                for vector in word_vector:
                    try:
                        vector.remove(word[idx])
                    except KeyError:
                        pass
        possible_words = match(word_vector, possible_words)
    

In [22]:
solve()

Attempt 1 with 2315 possible words
alter      | 0.39 
alert      | 0.39 
later      | 0.39 
arose      | 0.39 
irate      | 0.39 
stare      | 0.39 
arise      | 0.38 
raise      | 0.38 
renal      | 0.38 
learn      | 0.38 
saner      | 0.38 
snare      | 0.38 
steal      | 0.37 
least      | 0.37 
slate      | 0.37 
QA--QAQA True
Attempt 2 with 67 possible words
lathe      | 0.35 
petal      | 0.35 
metal      | 0.34 
fetal      | 0.34 
lithe      | 0.32 
untie      | 0.32 
octal      | 0.32 
satin      | 0.31 
bathe      | 0.31 
cutie      | 0.31 
patio      | 0.3  
extol      | 0.3  
setup      | 0.3  
fetus      | 0.29 
baton      | 0.29 
QA--QAQA True
Attempt 3 with 27 possible words
petal      | 0.35 
metal      | 0.34 
fetal      | 0.34 
octal      | 0.32 
extol      | 0.3  
setup      | 0.3  
fetus      | 0.29 
fetid      | 0.28 
vital      | 0.28 
until      | 0.27 
detox      | 0.27 
optic      | 0.26 
motif      | 0.23 
total      | 0.17 
titan      | 0.16 
QA--QAQA True
At